In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import backend as K
from matplotlib import pyplot as plt
%matplotlib inline
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
import helpers
from models import DaveModel, DaveVGG, DaveVGG19, SimpleModel, LeNetModel
from trainer import Trainer

Using TensorFlow backend.


In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
test = pd.read_json("_RawData/test.json/data/processed/test.json")

In [3]:
X = helpers.get_images(train)
X_test = helpers.get_images(test)

In [4]:
y = to_categorical(train.is_iceberg.values,num_classes=2)

In [5]:
# Xtr, Xv, ytr, yv = train_test_split(X, y, shuffle=False, test_size=0.20)

In [6]:
trainRunner = Trainer(X, y, [
    DaveModel(),
    DaveVGG(),
    DaveVGG19(),
    LeNetModel()
])

In [7]:
trainRunner.models[0].model.load_weights("davemodel20171203-181713.h5")
trainRunner.models[1].model.load_weights("vgg20171203-182704.h5")
trainRunner.models[2].model.load_weights("vgg1920171203-183725.h5")
trainRunner.models[3].model.load_weights("lenet20171203-184024.h5")

In [8]:
results = pd.DataFrame()

for model in trainRunner.models:
    name = model.get_name()
    print(name)
    test = model.model.predict(X_test)
    results[name] = test[:, 0]

davemodel
vgg
vgg19
lenet


In [9]:
results.head()

# results.iloc[:, 0]



,davemodel,vgg,vgg19,lenet
0,0.904778,0.506920,0.729676,0.523765
1,0.686788,0.277934,0.972092,0.611775
2,0.994779,0.248336,0.985353,0.991685
3,0.001994,0.011655,0.003505,0.004780
4,0.968177,0.280054,0.898017,0.604213


In [10]:
results["very_positive"] = results.apply(lambda x: x["davemodel"] > 0.5 and x["vgg"] > 0.5 and x["vgg19"] > 0.5 and x["lenet"] > 0.5, axis = 1)
results["very_negative"] = results.apply(lambda x: x["davemodel"] <= 0.5 and x["vgg"] <= 0.5 and x["vgg19"] <= 0.5 and x["lenet"] <= 0.5, axis = 1)
results["definitive"] = results["very_positive"] | results["very_negative"]

In [11]:
# results.head()
# results[results["definitive"]]
# results[results["very_positive"] != True & results["very_negative"] != True]

def get_score(value):
    score = 0
    
    if value > 0.5:
        score += 1
    else:
        score -= 1
        
    return score

def get_label(score):
    label = -1
    
    if score > 0:
        label = 1
    elif score < 0:
        label = 0
        
    return label

results["score"] = results.apply(lambda x: get_score(x["davemodel"]) + get_score(x["vgg"]) + get_score(x["vgg19"]) + get_score(x["lenet"]), axis = 1)

results["label"] = results.apply(lambda x: get_label(x["score"]), axis = 1)


In [12]:
X_train_pseudo = X_test[results["label"] >= 0]
y_train_pseudo = results[results["label"] >= 0]["label"]
X_train_pseudo.shape

(6984, 75, 75, 3)

In [13]:
ytr
results[results["label"] >= 0]["label"]

y_train_pseudo = np.array(pd.get_dummies(results[results["label"] >= 0]["label"]))
# y_train_pseudo = np.zeros((6984, 2))
y_train_pseudo
results[results["label"] >= 0]["label"]
# results.iloc[0]
# results[results["label"] >= 0]["label"]

test = []
stuff = results[results["label"] >= 0]["label"]

for record in stuff:
    if record == 0:
        test.append([0., 1.])
    elif record == 1:
        test.append([1., 0.])

y_train_pseudo = np.array(test)

In [14]:
print(X_train_pseudo.shape)
print(y_train_pseudo.shape)
print(Xtr.shape)
print(ytr.shape)

(6984, 75, 75, 3)
(6984, 2)
(1283, 75, 75, 3)
(1283, 2)


In [15]:
Xtr_p, Xv_p, ytr_p, yv_p = train_test_split(X_train_pseudo, y_train_pseudo, shuffle=False, test_size=0.20)

In [16]:
trainRunner = Trainer([
    DaveModel(Xtr_p, ytr_p, Xv_p, yv_p),
    DaveVGG(Xtr_p, ytr_p, Xv_p, yv_p),
    DaveVGG19(Xtr_p, ytr_p, Xv_p, yv_p),
    LeNetModel(Xtr_p, ytr_p, Xv_p, yv_p)
])

In [17]:
trainRunner.train(epoch = 50)

Model: davemodel
Batch Size: 32
Epochs: 50
Epoch 1/50
175/174 [==============================] - 9s - loss: 0.3322 - acc: 0.8443 - val_loss: 0.6399 - val_acc: 0.7130
Epoch 2/50
175/174 [==============================] - 7s - loss: 0.2456 - acc: 0.8905 - val_loss: 0.5322 - val_acc: 0.7373
Epoch 3/50
175/174 [==============================] - 7s - loss: 0.2094 - acc: 0.9063 - val_loss: 0.2100 - val_acc: 0.9055
Epoch 4/50
175/174 [==============================] - 7s - loss: 0.1850 - acc: 0.9214 - val_loss: 0.1649 - val_acc: 0.9291
Epoch 5/50
175/174 [==============================] - 7s - loss: 0.1698 - acc: 0.9238 - val_loss: 0.1522 - val_acc: 0.9377
Epoch 6/50
175/174 [==============================] - 7s - loss: 0.1676 - acc: 0.9266 - val_loss: 0.1367 - val_acc: 0.9356
Epoch 7/50
175/174 [==============================] - 7s - loss: 0.1512 - acc: 0.9333 - val_loss: 0.1475 - val_acc: 0.9413
Epoch 8/50
175/174 [==============================] - 7s - loss: 0.1513 - acc: 0.9342 - val_loss

175/174 [==============================] - 25s - loss: 0.1425 - acc: 0.9382 - val_loss: 0.1419 - val_acc: 0.9456
Epoch 17/50
175/174 [==============================] - 25s - loss: 0.1392 - acc: 0.9470 - val_loss: 0.1098 - val_acc: 0.9599
Epoch 18/50
175/174 [==============================] - 25s - loss: 0.1397 - acc: 0.9454 - val_loss: 0.1360 - val_acc: 0.9528
Epoch 19/50
175/174 [==============================] - 25s - loss: 0.1400 - acc: 0.9438 - val_loss: 0.1419 - val_acc: 0.9463
Epoch 20/50
175/174 [==============================] - 25s - loss: 0.1296 - acc: 0.9460 - val_loss: 0.1424 - val_acc: 0.9463
Epoch 21/50
175/174 [==============================] - 25s - loss: 0.1222 - acc: 0.9517 - val_loss: 0.1486 - val_acc: 0.9413
Epoch 22/50
175/174 [==============================] - 25s - loss: 0.1264 - acc: 0.9481 - val_loss: 0.1282 - val_acc: 0.9463
Epoch 23/50
175/174 [==============================] - 25s - loss: 0.1316 - acc: 0.9474 - val_loss: 0.1178 - val_acc: 0.9556
Epoch 24/50


175/174 [==============================] - 26s - loss: 0.1118 - acc: 0.9545 - val_loss: 0.1098 - val_acc: 0.9506
Epoch 32/50
175/174 [==============================] - 26s - loss: 0.1104 - acc: 0.9594 - val_loss: 0.0806 - val_acc: 0.9671
Epoch 33/50
175/174 [==============================] - 26s - loss: 0.1115 - acc: 0.9557 - val_loss: 0.0942 - val_acc: 0.9578
Epoch 34/50
175/174 [==============================] - 26s - loss: 0.1084 - acc: 0.9546 - val_loss: 0.0939 - val_acc: 0.9613
Epoch 35/50
175/174 [==============================] - 26s - loss: 0.1027 - acc: 0.9614 - val_loss: 0.1160 - val_acc: 0.9635
Epoch 36/50
175/174 [==============================] - 26s - loss: 0.1046 - acc: 0.9588 - val_loss: 0.0919 - val_acc: 0.9628
Epoch 37/50
175/174 [==============================] - 26s - loss: 0.1004 - acc: 0.9587 - val_loss: 0.0963 - val_acc: 0.9599
Epoch 38/50
175/174 [==============================] - 26s - loss: 0.0943 - acc: 0.9624 - val_loss: 0.0789 - val_acc: 0.9656
Epoch 39/50


175/174 [==============================] - 7s - loss: 0.1047 - acc: 0.9574 - val_loss: 0.1137 - val_acc: 0.9477
Epoch 47/50
175/174 [==============================] - 6s - loss: 0.1129 - acc: 0.9517 - val_loss: 0.0976 - val_acc: 0.9606
Epoch 48/50
175/174 [==============================] - 7s - loss: 0.1126 - acc: 0.9541 - val_loss: 0.0954 - val_acc: 0.9585
Epoch 49/50
175/174 [==============================] - 6s - loss: 0.1177 - acc: 0.9529 - val_loss: 0.1001 - val_acc: 0.9621
Epoch 50/50
175/174 [==============================] - 7s - loss: 0.1076 - acc: 0.9573 - val_loss: 0.1056 - val_acc: 0.9613


In [19]:
from keras.preprocessing.image import ImageDataGenerator


# After this, let's try and predict the non-definitive data, and see if it's now definitive
predict = []
# X_test = helpers.get_images(test)
blah = X_test[results["label"] < 0]

for model in trainRunner.models:
    pred_gen = ImageDataGenerator()
    predict.append(model.model.predict_generator(pred_gen.flow(blah, batch_size=32, shuffle = False), len(blah) / 32))

#     if submit:
#         self.create_submission(predict, test)
#     predict.append(model.predict(blah))
    
# Also, do a predict and submit



In [23]:
results2 = pd.DataFrame()

for model2 in trainRunner.models:
    name = model2.get_name()
    print(name)
    test2 = model2.model.predict(blah)
    results2[name] = test2[:, 0]

davemodel
vgg
vgg19
lenet


In [24]:
results2.head()

,davemodel,vgg,vgg19,lenet
0,0.977051,0.525496,0.897786,0.504032
1,0.753548,0.285678,0.986055,0.647878
2,0.237472,0.007978,0.062903,0.004565
3,0.901699,0.690666,0.516560,0.125253
4,0.182136,0.147875,0.963147,0.535211


In [25]:
results2["very_positive"] = results2.apply(lambda x: x["davemodel"] > 0.5 and x["vgg"] > 0.5 and x["vgg19"] > 0.5 and x["lenet"] > 0.5, axis = 1)
results2["very_negative"] = results2.apply(lambda x: x["davemodel"] <= 0.5 and x["vgg"] <= 0.5 and x["vgg19"] <= 0.5 and x["lenet"] <= 0.5, axis = 1)
results2["definitive"] = results2["very_positive"] | results2["very_negative"]

In [28]:
print(results2.shape)

results2[results2["definitive"]]

(1440, 7)


,davemodel,vgg,vgg19,lenet,very_positive,very_negative,definitive
0,0.977051,0.525496,0.897786,0.504032,True,False,True
2,0.237472,0.007978,0.062903,0.004565,False,True,True
7,0.999999,0.999595,1.000000,0.950964,True,False,True
8,0.999982,0.997978,1.000000,0.999996,True,False,True
9,1.000000,0.999806,1.000000,0.999868,True,False,True
10,0.929262,0.813650,0.999986,0.959610,True,False,True
11,0.974176,0.870007,0.997253,0.935104,True,False,True
12,0.987627,0.653146,0.647587,0.809050,True,False,True
14,0.999294,0.998570,1.000000,0.999980,True,False,True
15,1.000000,0.999618,1.000000,0.997085,True,False,True
